In [1]:
from datetime import datetime, timedelta

# Set the symbol and timeframe
symbol = 'BTC/USDT:USDT'
timeframe = '5m'

# Calculate the start and end times
end_time = datetime.now()
start_time = end_time - timedelta(days=1)

# Convert timestamps to milliseconds
since = int(start_time.timestamp() * 1000)
end = int(end_time.timestamp() * 1000)

In [5]:
import ccxt
from pandas import DataFrame, to_datetime

# Initialize the Binance Futures exchange
exchange = ccxt.binanceusdm()

# Fetch the OHLCV data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit=None)

# Convert the data to a pandas DataFrame
ohlcv = DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert timestamp to datetime
ohlcv['timestamp'] = to_datetime(ohlcv['timestamp'], unit='ms')

ohlcv.set_index('timestamp', inplace=True)

ohlcv.head()

,open,high,low,close,volume
timestamp,,,,,
2024-10-03 04:15:00,61302.1,61354.1,61287.0,61335.4,250.193
2024-10-03 04:20:00,61335.4,61335.4,61272.0,61312.4,265.529
2024-10-03 04:25:00,61312.5,61347.0,61252.9,61329.1,591.848
2024-10-03 04:30:00,61329.1,61418.0,61325.0,61388.5,405.865
2024-10-03 04:35:00,61388.5,61397.3,61351.3,61393.0,160.115


In [34]:
from src.transformers.ohlcv.market_structure import detect_pivot_points

# Apply the pivot point detection to our OHLCV data
ohlcv['pivot'] = detect_pivot_points(ohlcv, window=3)

In [50]:
import plotly.graph_objects as go

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=ohlcv.index,
                open=ohlcv['open'],
                high=ohlcv['high'],
                low=ohlcv['low'],
                close=ohlcv['close'])])

# Add pivot points to the chart
for pivot_type in ['HH', 'LH', 'HL', 'LL']:
    pivot_data = ohlcv[ohlcv['pivot'] == pivot_type]
    if pivot_type in ['HH', 'LH']:
        y = pivot_data['high']
        marker_color = 'lime' if pivot_type == 'HH' else 'pink'
        text_pos = 'top center'
    else:
        y = pivot_data['low']
        marker_color = 'red' if pivot_type == 'LL' else 'green'
        text_pos = 'bottom center'
    
    fig.add_trace(go.Scatter(
        x=pivot_data.index,
        y=y,
        mode='markers+text',
        name=pivot_type,
        text=[pivot_type] * len(pivot_data),
        textposition=text_pos,
        marker=dict(color=marker_color, size=8),
        textfont=dict(size=10)
    ))


# Update layout
fig.update_layout(
    title=f'{symbol} OHLCV Chart with Pivot Points',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the plot
fig.show()

In [51]:
from src.transformers.ohlcv.market_structure import detect_market_structure_breaks

# Apply the function to detect market structure breaks
ohlcv = detect_market_structure_breaks(ohlcv, window=6)

# Update the plot to include BOS points
bos_highs_data = ohlcv[ohlcv['BOS'] & (ohlcv['pivot'].isin(['HH', 'LH']))]

fig.add_trace(go.Scatter(
    x=bos_highs_data.index,
    y=bos_highs_data['high'],
    mode='markers',
    name='BOS (High)',
    marker=dict(color='yellow', size=12, symbol='star'),
))

bos_lows_data = ohlcv[ohlcv['BOS'] & (ohlcv['pivot'].isin(['HL', 'LL']))]

fig.add_trace(go.Scatter(
    x=bos_lows_data.index,
    y=bos_lows_data['low'],
    mode='markers',
    name='BOS (Low)',
    marker=dict(color='yellow', size=12, symbol='star-triangle-down'),
))

# Update layout
fig.update_layout(
    title=f'{symbol} OHLCV Chart with Pivot Points and BOS',
)

# Show the updated plot
fig.show()
